In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# ---------------------------
# Load all input CSVs
# ---------------------------
team_df = pd.read_csv('team.csv')
budgets_df = pd.read_csv('budgets.csv')
holidays_df = pd.read_csv('holidays.csv')
leaves_df = pd.read_csv('leaves.csv')
logged_df = pd.read_csv('logged.csv')

# Clean headers
for df in [budgets_df, holidays_df, leaves_df, logged_df]:
    df.columns = df.columns.str.strip()

In [3]:
# ---------------------------
# Define reporting period
# ---------------------------
from_date = 20250701
to_date = 20250716

from_dt = datetime.strptime(str(from_date), "%Y%m%d")
to_dt = datetime.strptime(str(to_date), "%Y%m%d")
total_days = (to_dt - from_dt).days + 1

In [4]:
# ---------------------------
# Calculate holidays count
# ---------------------------
num_holidays = holidays_df[
    (holidays_df['day'] >= from_date) & (holidays_df['day'] <= to_date)
].shape[0]

In [5]:
# ---------------------------
# Build base report with summarized leaves
# ---------------------------
report_df = team_df.merge(leaves_df, on='id', how='left').fillna(0)
report_df['leaves'] = report_df['leaves'].astype(int)
report_df['holidays'] = num_holidays
report_df['total_days'] = total_days
report_df['work_days'] = report_df['total_days'] - report_df['holidays'] - report_df['leaves']

In [6]:
# ---------------------------
# Add original logged days
# ---------------------------
logged_days = logged_df.groupby('id')['logged'].sum().reset_index(name='logged_days')
report_df = report_df.merge(logged_days, on='id', how='left').fillna(0)

# Pivot logged tasks
task_pivot = logged_df.pivot_table(index='id', columns='task', values='logged', aggfunc='sum').reset_index().fillna(0)
report_df = report_df.merge(task_pivot, on='id', how='left')

# Ensure all tasks exist & fill NaN
for task in ['task1', 'task2', 'task3', 'task4']:
    if task not in report_df.columns:
        report_df[task] = 0.0
report_df[['task1', 'task2', 'task3', 'task4']] = report_df[['task1', 'task2', 'task3', 'task4']].fillna(0)

# Add total original logged
report_df['total_original_logged'] = report_df[['task1', 'task2', 'task3', 'task4']].sum(axis=1)

# Compute balance_days
report_df['balance_days'] = report_df['work_days'] - report_df['logged_days']

In [7]:
# ---------------------------
# Calculate total team capacity & task budgets
# ---------------------------
team_size = report_df.shape[0]
hours_per_day = 8

total_team_work_days = report_df['work_days'].sum()
total_team_work_hours = total_team_work_days * hours_per_day

task_budgets_days = {}
task_budgets_hours = {}
task_totals = {}

print("\n=== Team Capacity ===")
print(f"Team members: {team_size}")
print(f"Total team capacity: {total_team_work_days:.2f} days | {total_team_work_hours:.2f} hours")

print("\n=== Task Budgets (FTE → Days & Hours) ===")
total_team_fte = team_size * 1

for _, row in budgets_df.iterrows():
    task = row['task']
    task_fte = row['budget']

    task_pct = task_fte / total_team_fte
    budget_days = total_team_work_days * task_pct
    budget_hours = total_team_work_hours * task_pct

    task_budgets_days[task] = budget_days
    task_budgets_hours[task] = budget_hours
    task_totals[task] = report_df[task].sum()

    print(f"{task}: {task_fte} FTE ({task_pct*100:.2f}%) → {budget_days:.2f} days | {budget_hours:.2f} hrs")


=== Team Capacity ===
Team members: 19
Total team capacity: 211.00 days | 1688.00 hours

=== Task Budgets (FTE → Days & Hours) ===
task1: 0.23 FTE (1.21%) → 2.55 days | 20.43 hrs
task2: 2.82 FTE (14.84%) → 31.32 days | 250.53 hrs
task3: 3.95 FTE (20.79%) → 43.87 days | 350.93 hrs
task4: 12.0 FTE (63.16%) → 133.26 days | 1066.11 hrs


In [8]:
# ---------------------------
# Calculate task net remaining budget
# ---------------------------
task_remaining_budget = {}
for task in ['task1', 'task2', 'task3', 'task4']:
    allowed = task_budgets_days[task]
    used = report_df[task].sum()
    task_remaining_budget[task] = max(0, allowed - used)

print("\n=== Remaining Team Budget per Task (Days) ===")
print(task_remaining_budget)


=== Remaining Team Budget per Task (Days) ===
{'task1': np.float64(2.5542105263157895), 'task2': np.float64(31.316842105263152), 'task3': np.float64(38.43578947368421), 'task4': np.float64(127.31315789473682)}


In [9]:
# ---------------------------
# Total leftover pool
# ---------------------------
total_leftover_pool = report_df['balance_days'].sum()
print(f"\nTotal leftover balance_days to allocate: {total_leftover_pool:.2f}")

# ---------------------------
# Add columns for pooled allocation
# ---------------------------
for task in ['task1', 'task2', 'task3', 'task4']:
    report_df[f'log_{task}'] = 0.0


Total leftover balance_days to allocate: 199.62


In [10]:
# ---------------------------
# Pooled-proportional allocation
# ---------------------------
np.random.seed(42)

for idx, row in report_df.iterrows():
    leftover = row['balance_days']
    if leftover <= 0:
        continue

    # Use current remaining budget for tasks
    task_weights = np.array([task_remaining_budget[task] for task in ['task1','task2','task3','task4']])
    if task_weights.sum() == 0:
        continue  # no room left anywhere

    task_weights_normalized = task_weights / task_weights.sum()

    for i, task in enumerate(['task1','task2','task3','task4']):
        portion = round(leftover * task_weights_normalized[i], 2)
        assignable = min(portion, task_remaining_budget[task])
        report_df.at[idx, f'log_{task}'] += assignable
        task_remaining_budget[task] -= assignable

    # Final leftover for this ID
    new_total_logged = report_df.loc[idx, ['log_task1','log_task2','log_task3','log_task4']].sum()
    report_df.at[idx, 'total_logged'] = new_total_logged
    report_df.at[idx, 'leftover_balance_days'] = row['balance_days'] - new_total_logged

In [11]:
# ---------------------------
# Sort by ID
# ---------------------------
report_df = report_df.sort_values(by='id').reset_index(drop=True)

# ---------------------------
# Final columns
# ---------------------------
final_cols = [
    'id', 'name', 'leaves', 'holidays', 'total_days', 'work_days',
    'logged_days', 'task1', 'task2', 'task3', 'task4', 'total_original_logged',
    'balance_days', 'log_task1', 'log_task2', 'log_task3', 'log_task4',
    'total_logged', 'leftover_balance_days'
]

report_df = report_df[final_cols]

In [12]:
# ---------------------------
# Final output
# ---------------------------
print("\n=== Final Team Allocation Report ===")
report_df


=== Final Team Allocation Report ===


,id,name,leaves,holidays,total_days,work_days,logged_days,task1,task2,task3,task4,total_original_logged,balance_days,log_task1,log_task2,log_task3,log_task4,total_logged,leftover_balance_days
0,p001,aaa,0,4,16,12,5.89,0.0,0.0,3.10,2.79,5.89,6.11,0.08,0.960000,1.180000,3.900000,6.120000,-1.000000e-02
1,p002,bbb,4,4,16,8,0.00,0.0,0.0,0.00,0.00,0.00,8.00,0.10,1.260000,1.540000,5.100000,8.000000,0.000000e+00
2,p003,ccc,2,4,16,10,0.01,0.0,0.0,0.00,0.01,0.01,9.99,0.13,1.570000,1.920000,6.370000,9.990000,0.000000e+00
3,p004,ddd,2,4,16,10,0.00,0.0,0.0,0.00,0.00,0.00,10.00,0.13,1.570000,1.930000,6.380000,10.010000,-1.000000e-02
4,p005,eee,3,4,16,9,0.01,0.0,0.0,0.00,0.01,0.01,8.99,0.11,1.410000,1.730000,5.730000,8.980000,1.000000e-02
5,p006,fff,3,4,16,9,0.00,0.0,0.0,0.00,0.00,0.00,9.00,0.12,1.410000,1.730000,5.740000,9.000000,0.000000e+00
6,p007,ggg,0,4,16,12,2.50,0.0,0.0,0.17,2.33,2.50,9.50,0.12,1.490000,1.830000,6.060000,9.500000,0.000000e+00
7,p008,hhh,0,4,16,12,2.49,0.0,0.0,2.13,0.36,2.49,9.51,0.12,1.490000,1.830000,6.070000,9.510000,0.000000e+00
8,p009,iii,2,4,16,10,0.08,0.0,0.0,0.00,0.08,0.08,9.92,0.13,1.560000,1.910000,6.330000,9.930000,-1.000000e-02
9,p010,jjj,1,4,16,11,0.01,0.0,0.0,0.00,0.01,0.01,10.99,0.14,1.720000,2.120000,7.010000,10.990000,0.000000e+00


In [13]:
# ---------------------------
# Final Task Totals
# ---------------------------
print("\n=== Final Task Totals (Original + Balanced) vs Budget ===")
for task in ['task1', 'task2', 'task3', 'task4']:
    final_total_days = report_df[task].sum() + report_df[f'log_{task}'].sum()
    final_total_hours = final_total_days * hours_per_day
    budget_days = task_budgets_days[task]
    budget_hours = task_budgets_hours[task]

    status = "✅ OK" if final_total_days <= budget_days + 0.01 else "⚠️ OVERBOOKED"
    print(f"{task}: {final_total_days:.2f} days ({final_total_hours:.2f} hrs) "
          f"/ {budget_days:.2f} days ({budget_hours:.2f} hrs) | {status}")



=== Final Task Totals (Original + Balanced) vs Budget ===
task1: 2.55 days (20.40 hrs) / 2.55 days (20.43 hrs) | ✅ OK
task2: 31.32 days (250.53 hrs) / 31.32 days (250.53 hrs) | ✅ OK
task3: 43.87 days (350.93 hrs) / 43.87 days (350.93 hrs) | ✅ OK
task4: 133.26 days (1066.11 hrs) / 133.26 days (1066.11 hrs) | ✅ OK


In [14]:
# ---------------------------
# Final leftover summary
# ---------------------------
print("\n=== Summary ===")
total_balance = report_df['balance_days'].sum()
total_allocated = report_df['total_logged'].sum()
total_leftover = report_df['leftover_balance_days'].sum()

print(f"Total team balance_days: {total_balance:.2f}")
print(f"Total allocated during balancing: {total_allocated:.2f}")
print(f"Total leftover after balancing: {total_leftover:.2f}")


=== Summary ===
Total team balance_days: 199.62
Total allocated during balancing: 199.62
Total leftover after balancing: 0.00
